In [1]:
import pickle

# File path of the saved pickle
file_path = r"F:\1.Image_List\Flowers102_400\combo_list.pickle"

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")


Combo list loaded successfully.


In [55]:
from sklearn.preprocessing import LabelEncoder
y = []
for i in range(0,len(loaded_combo_list)):
    y.append(i)
categorical_labels = to_categorical(y)


In [56]:
from glob import glob
k = 0
train=[]
tr_y=[]

for i in loaded_combo_list[:360]:
    x = glob(i+'/*')
    train.append(x)
    for j in x:
        tr_y.append(categorical_labels[k])
    k = k + 1
    
train = np.concatenate(train)

k = 0
val=[]
va_y=[]

for i in loaded_combo_list[360:]:
    x = glob(i+'/*')
    val.append(x)
    for j in x:
        va_y.append(categorical_labels[k])
    k = k + 1
val = np.concatenate(val)

In [66]:
from tensorflow.keras.utils import Sequence
from PIL import Image
import os
target_size = (224,224)

def get_te(k , a) :
    x = glob(k+'/*')
    #print("..........................")
    #print(x)
    #print("..........................")
    img = Image.open(k).resize(target_size)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    idata = np.array(img)
    X_train = idata.astype('float32') / 255.
    #print("..........................")
    #print(np.shape(X_train))
    #print("..........................")
    return X_train

def get_cat(k) :
    return np.array(k)

class My_Custom_Generator(Sequence) :
  
  def __init__(self, filename , labels, batch_size) :
    self.filename = filename
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    y_train = get_cat(batch_y)
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x]), np.array( y_train )


class My_Test_Generator(Sequence) :
  
  def __init__(self, filename , batch_size) :
    self.filename = filename
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x])

batch_size = 100
my_training_batch_generator = My_Custom_Generator(train, tr_y, batch_size)
my_validation_batch_generator = My_Custom_Generator(val, va_y, batch_size)

In [51]:
#batch_images, batch_labels = my_training_batch_generator.__getitem__(0)
#batch_labels

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


# Load the pre-trained ResNet50 model (excluding the top layer)
base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224 , 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(512, activation='relu')(x)

# Add the output layer with the number of classes
output = Dense(400, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Define the optimizer with SGD and momentum
optimizer = SGD(lr=0.001, momentum=0.9)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(generator=my_training_batch_generator, validation_data=my_validation_batch_generator, epochs=100)

C:\Users\shaif\AppData\Local\Temp\ipykernel_2208\3146873370.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=my_training_batch_generator, validation_data=my_validation_batch_generator, epochs=100)
C:\Users\shaif\AppData\Local\Temp\ipykernel_2208\2899005922.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/100
  28/3972 [..............................] - ETA: 1:47:12 - loss: 6.9284 - accuracy: 0.0000e+00

In [17]:
np.shape(x_train)

(445349, 32, 32, 3)

Epoch 1/100
5567/5567 [==============================] - 428s 76ms/step - loss: 4.9830 - accuracy: 0.0293 - val_loss: 11.0171 - val_accuracy: 0.0000e+00
Epoch 2/100
5567/5567 [==============================] - 410s 74ms/step - loss: 4.2783 - accuracy: 0.0526 - val_loss: 11.8687 - val_accuracy: 2.2454e-05
Epoch 3/100
5567/5567 [==============================] - 408s 73ms/step - loss: 3.7722 - accuracy: 0.0724 - val_loss: 13.0242 - val_accuracy: 0.0000e+00
Epoch 4/100
5567/5567 [==============================] - 409s 74ms/step - loss: 3.3515 - accuracy: 0.0913 - val_loss: 13.9015 - val_accuracy: 0.0000e+00
Epoch 5/100
5567/5567 [==============================] - 408s 73ms/step - loss: 3.0421 - accuracy: 0.1083 - val_loss: 14.4977 - val_accuracy: 0.0000e+00
Epoch 6/100
5567/5567 [==============================] - 467s 84ms/step - loss: 2.8128 - accuracy: 0.1210 - val_loss: 15.0699 - val_accuracy: 8.9817e-05
Epoch 7/100
5567/5567 [==============================] - 581s 104ms/step - loss: 2

In [ ]:
model.save('D:/Flowers102_224_model.h5')